In [2]:
import sys
sys.path.append('structural-probes')
from pathlib import Path

from run_experiment import setup_new_experiment_dir, execute_experiment
import yaml
import torch
import pandas as pd
import eval_probes_on_dataset

In [3]:
data_folder = Path("../../..")
cola = data_folder / 'data'/ 'SST-2' / 'original' / 'raw'

In [15]:
def setup_args_and_folder(): 
    CONFIG_FILE = 'example/config/bert_ptb3.yaml'
    EXPERIMENT_NAME = ''
    SEED = 123

    class Object(object):
        pass

    cli_args = Object()
    cli_args.experiment_config = CONFIG_FILE
    cli_args.results_dir = EXPERIMENT_NAME
    cli_args.train_probe = -1
    cli_args.report_results = 1
    cli_args.seed = SEED

    yaml_args = yaml.load(open(cli_args.experiment_config))
    setup_new_experiment_dir(cli_args, yaml_args, cli_args.results_dir)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    yaml_args['device'] = device
    return yaml_args

yaml_args = setup_args_and_folder()

Constructing new results directory at results/BERT-disk-parse-distance-2020-4-26-11-2-34-35823/


/home/mli/anaconda3/envs/data-mining/lib/python3.6/site-packages/ipykernel_launcher.py:16: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  app.launch_new_instance()


# Notes for generating the Tree data for SST
- This is for reading in the Ground Truth trees that is already given to us in SST
- We'll use Stanford's CoreNLP tools
- Run the ReadSentimentDataset `java -mx4g edu.stanford.nlp.sentiment.ReadSentimentDataset -inputDir data/SST-2/original -outputDir tmp/`
  - The ground truth already does subword partitions, so need to account for that

In [16]:
from nltk.tree import Tree

from reporter import WordPairReporter, WordReporter, prims_matrix_to_edges
from tqdm import tqdm
import copy

In [17]:
def read_trees(path):
    with open(path) as f:
        tree_lines = f.readlines()
        
    trees = [Tree.fromstring(treeline) for treeline in tree_lines]
    return trees

def read_sentiment_sentences(path):
    with open(path) as f:
        all_sentences = f.readlines()
        
        sentences, labels = [], []
        for pair in all_sentences:
            sentence, label = pair.split('\t')
            sentences.append(sentence)
            labels.append(label)
        return sentences, labels

In [43]:
data_base = Path('../../../data/SST-2')

train_path = data_base / 'sentence_splits' / 'train.tsv'
dev_path = data_base / 'sentence_splits' / 'dev.tsv'

# read in SST dataset
sst_trees_base = data_base / 'tree_format/'
gt_tree_train_path = sst_trees_base / 'train.txt'
gt_tree_dev_path = sst_trees_base / 'dev.txt'

train_sentiment, train_labels = read_sentiment_sentences(train_path)
dev_sentiment, dev_labels = read_sentiment_sentences(dev_path)

# Read into NTLK Trees
gt_train_trees = read_trees(gt_tree_train_path)
gt_dev_trees = read_trees(gt_tree_dev_path)

# Calculate distance between the two trees
# one tree's format is from structural probes
# other tree's format is from the dataset (gt_*_trees)

In [45]:
print(len(gt_dev_trees), len(dev_sentiment))

1101 1101


In [53]:
import importlib
importlib.reload(eval_probes_on_dataset)
# this is paired with parse-distance
pair_reporter = WordPairReporter(yaml_args) 

# this is paired with parse-depth
reporter = WordReporter(yaml_args)

word_dists, word_depths, predicted_edges = eval_probes_on_dataset.report_on_stdin(yaml_args, dev_sentiment)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


[demoing]: 0it [00:00, ?it/s]

[demoing]: 5it [00:00, 41.80it/s]

Constructing TwoWordPSDProbe
Constructing OneWordPSDProbe
prediction edges 11
prediction edges 10
prediction edges 21
prediction edges 4
prediction edges 20
prediction edges 11
prediction edges 5




[demoing]: 9it [00:00, 40.31it/s]

[demoing]: 14it [00:00, 40.35it/s]

prediction edges 22
prediction edges 26
prediction edges 23
prediction edges 25
prediction edges 26
prediction edges 18
prediction edges 26
prediction edges 10
prediction edges 20




[demoing]: 19it [00:00, 40.86it/s]

[demoing]: 24it [00:00, 41.13it/s]

...

prediction edges 8
prediction edges 13
prediction edges 16
prediction edges 18
prediction edges 14
prediction edges 8
prediction edges 18
prediction edges 4
prediction edges 3
prediction edges 8




[demoing]: 1088it [00:25, 44.08it/s]

[demoing]: 1093it [00:25, 43.90it/s]

prediction edges 13
prediction edges 22
prediction edges 19
prediction edges 27
prediction edges 21
prediction edges 19
prediction edges 5
prediction edges 21
prediction edges 23




[demoing]: 1101it [00:25, 43.01it/s]

prediction edges 17
prediction edges 20
prediction edges 23
prediction edges 28
prediction edges 16
prediction edges 21
prediction edges 25


In [48]:
def earliest_root(a, b):
    for i in range(min(len(a), len(b))):
        if a[i] != b[i]:
            return i
    return -1

def get_adj_matrix(cur_tree):
    num_words = len(cur_tree.leaves())
    all_paths = [cur_tree.leaf_treeposition(i) for i in range(num_words)]
    matrix = [[0] * num_words for _ in range(num_words)]

    for a_idx, a in enumerate(all_paths):
        for b_idx, b in enumerate(all_paths):
            if a_idx < b_idx:
                root = earliest_root(a, b)
                matrix[a_idx][b_idx] = len(a) + len(b) - 2 * root
                matrix[b_idx][a_idx] = len(a) + len(b) - 2 * root
    return matrix

In [56]:
def report_uuas(predicted_edges, trees, split_name):
    uspan_total = 0
    uspan_correct = 0
    total_sents = 0
    for predicted_edge, gt_tree in tqdm(zip(predicted_edges, trees), desc='[uuas,tikz]'):
        words = gt_tree.leaves()
        poses = gt_tree.leaves()
        gold_edges = get_adj_matrix(gt_tree)
        gold_edges = prims_matrix_to_edges(gold_edges, words, poses)
        pred_edges = predicted_edge
#         pred_edges = prims_matrix_to_edges(prediction_batch, words, poses)

#         if split_name != 'test' and total_sents < 20:
#             self.print_tikz(pred_edges, gold_edges, words, split_name)

#         print(total_sents)
#         print(gold_edges)
#         print(pred_edges)
#         print()
#         import pdb
#         pdb.set_trace()
        gold_span = set([tuple(sorted(x)) for x in gold_edges])
        pred_span = set([tuple(sorted(x)) for x in pred_edges])
        uspan_correct += len(gold_span.intersection(pred_span))
        uspan_total += len(gold_edges)
        total_sents += 1

    uuas = uspan_correct / float(uspan_total)
    return uuas

print(report_uuas(predicted_edges, gt_dev_trees, 'dev'))



[uuas,tikz]: 0it [00:00, ?it/s]

[uuas,tikz]: 57it [00:00, 566.86it/s]

[uuas,tikz]: 117it [00:00, 576.38it/s]

[uuas,tikz]: 179it [00:00, 586.17it/s]

[uuas,tikz]: 234it [00:00, 574.34it/s]

[uuas,tikz]: 297it [00:00, 588.18it/s]

[uuas,tikz]: 370it [00:00, 624.00it/s]

[uuas,tikz]: 434it [00:00, 626.75it/s]

[uuas,tikz]: 509it [00:00, 658.03it/s]

[uuas,tikz]: 573it [00:00, 620.84it/s]

[uuas,tikz]: 637it [00:01, 625.83it/s]

[uuas,tikz]: 706it [00:01, 642.15it/s]

[uuas,tikz]: 770it [00:01, 633.56it/s]

[uuas,tikz]: 834it [00:01, 618.82it/s]

[uuas,tikz]: 896it [00:01, 610.05it/s]

[uuas,tikz]: 966it [00:01, 631.37it/s]

[uuas,tikz]: 1030it [00:01, 628.97it/s]

[uuas,tikz]: 1101it [00:01, 628.20it/s]

0.4618542972868002
